In [12]:
## Load the dataset and return the first few rows
import glob
import pandas as pd
pd.options.display.max_columns = None
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,8)

Set up a regex to find the year from the filename

In [13]:
import re
year_match = re.compile('(\d{4})')

Couple of functions to transform the scoreline into useful data

In [14]:
def goals(result: str):
    '''Given a scoreline such as 6-1 returns home and away goals in a dict {home: 6, away: 1}'''
    nums = result.split('-')
    try:
        return {'home': int(nums[0]), 'away': int(nums[1])}
    except Exception:
        return {'home': -1, 'away': -1}

In [15]:
def outcome(result):
    '''Given a string of the result e.g. 6-1 categorise as either a no-score draw (1), score draw (2), home win (3) or away win (4)'''
    goals_result = goals(result)
    if goals_result['home'] == goals_result['away']:
        if goals_result['home'] == 0:
            return 1
        else:
            return 2
    elif goals_result['home'] > goals_result['away']:
        return 3
    else:
        return 4

Generate a list of available leagues from the folder names

In [16]:
league_list = glob.glob('/Users/dev/aicore/Football-predictor/Football-Dataset/*')
league_list = pd.DataFrame(league_list)
league_list.columns = ['folder']
league_list['name'] = league_list['folder'].apply(lambda x: x.split('/')[-1])

Get all files in a particular folder and list their names

In [17]:
def get_files(folder):
    league_file_list = glob.glob(folder + '/*')
    league_file_list.sort()
    return league_file_list
league_list['files'] = league_list['folder'].apply(get_files)

Now run through the list of leagues (folders) and the files in them (in the files column), read in the data, and combine them together into dfs.

Results:

- `overall_team_counts` is a df showing the number of teams involved per league per season (unlikely to be consistent - and indeed not - EPL is a counterexample)
- `df_full` is the whole data of results across all leagues and seasons

In [18]:
overall_team_counts = pd.DataFrame()
df_league_lists = []
for index, league in league_list.iterrows():
    league_file_list = league['files']
    team_counts = {}
    df_list = []
    for file in league_file_list:
        df = pd.read_csv(file)
        df.columns = df.columns.str.lower()
        year = year_match.findall(file)[0]
        if len(df.index) == 0:
            # print('No data for {}'.format(year))
            pass
        else:
            df_original = df.copy()
            df_list.append(df)
            team_counts[year] = len(df['home_team'].unique())
            # print('In {} there were {} teams'.format(year, len(df['Home_Team'].unique())))
    overall_team_counts = pd.concat([overall_team_counts, pd.Series(team_counts, name=league['name'])], axis=1)
    df_league_lists.append(pd.concat(df_list, ignore_index=True))
df_full = pd.concat(df_league_lists, ignore_index=True)
overall_team_counts.sort_index()
overall_team_counts = overall_team_counts.fillna(0).astype(int)

Use the functions defined earlier - `goals` and `outcome` - to transform the scoreline (as a string) into numerical data

In [19]:
df_full['goals'] = df_full['result'].apply(goals)
# in case we're repeating this operation
try:
    df_full = df_full.drop(['home','away'],axis=1) 
except Exception:
    pass
df_full = pd.concat([df_full, pd.json_normalize(df_full['goals'])], axis=1).drop('goals', axis=1)
# drop errors
df_full = df_full.drop(df_full[df_full.home == -1].index)
df_full['total_goals'] = df_full['home'] + df_full['away']
df_full['outcome'] = df_full['result'].apply(outcome)

In [20]:
df = df_full.copy()

## Additional datasets - join them on

## first: match info

In [21]:
match_df = pd.read_csv('additional-data/Match_Info.csv')
match_df.columns = match_df.columns.str.lower()
# match_df.info()
match_df['referee'] = match_df['referee'].str.strip().str.replace('Referee: ','')
match_df['date_dt'] = pd.to_datetime(match_df['date_new'])
match_df['link_parts'] = match_df['link'].str.split('/')
match_df['home_team'] = match_df.apply(lambda row: row['link_parts'][-3], axis=1)
match_df['away_team'] = match_df.apply(lambda row: row['link_parts'][-2], axis=1)
match_df['season'] = match_df.apply(lambda row: row['link_parts'][-1], axis=1)
match_df = match_df.drop(['date_new'], axis=1)

In [22]:
import difflib
df_teams = df['home_team'].unique()
match_teams = match_df['home_team'].unique()

Let's use Levenshtein. see https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [23]:
from fuzzywuzzy import process
# needs a tweak for Manchester to give a helping hand on those two teams
matched_teams = [(d,(process.extractOne(d.replace('Man.','Manchester'), match_teams))[0]) for d in df_teams]

In [24]:
def team_name_for_link(t):
    matches = [x[1] for x in matched_teams if x[0] == t]
    return (matches if len(matches)>0 else [''])[0]
df['link'] = df.apply(lambda row: '/'.join(['','match',team_name_for_link(row['home_team']),team_name_for_link(row['away_team']),str(row['season'])]).strip().lower().replace(' ','-'), axis=1)

In [25]:
# duplicates?
print(df['link'].duplicated().value_counts())
print(df[['home_team','away_team','season','league']].duplicated().value_counts())
df.drop_duplicates(subset=['home_team','away_team','season','league'], inplace=True)

False    130890
True      15602
Name: link, dtype: int64
False    132103
True      14389
dtype: int64


In [26]:
df2 = pd.merge(match_df, df, how="inner", on='link')

## add team info

In [27]:
team_df = pd.read_csv('additional-data/Team_Info.csv')
team_df.columns = team_df.columns.str.lower()
team_df['capacity'] = pd.to_numeric(team_df['capacity'].str.replace(',',''))
team_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544 entries, 0 to 543
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   team      544 non-null    object
 1   city      544 non-null    object
 2   country   544 non-null    object
 3   stadium   447 non-null    object
 4   capacity  544 non-null    int64 
 5   pitch     447 non-null    object
dtypes: int64(1), object(5)
memory usage: 25.6+ KB


In [28]:
teams_from_team_data = set(team_df.team)
link_team_data = set(df2.home_team_x)
# originally I had this the other way round, which resulted in multiple matches
matched_teams_team_data = [(d,(process.extractOne(d.replace('Man.','Manchester'), teams_from_team_data))[0]) for d in link_team_data]
matched_teams_team_data

[('msv-duisburg', 'MSV Duisburg'),
 ('betis', 'Real Betis'),
 ('pordenone', 'Pordenone'),
 ('fc-remscheid', 'FC Remscheid'),
 ('coventry-city', 'Coventry City'),
 ('real-murcia', 'Real Murcia'),
 ('fc-martigues', 'FC Martigues'),
 ('orihuela', 'Orihuela CF'),
 ('fortuna-dusseldorf', 'Fortuna'),
 ('hamburger-sv', 'Hamburger SV'),
 ('arouca', 'Arouca'),
 ('ciudad-de-murcia', 'Ciudad de Murcia'),
 ('vfb-oldenburg', 'VfB Oldenburg'),
 ('leicester-city-fc', 'Leicester'),
 ('albacete', 'Albacete'),
 ('treviso', 'Treviso'),
 ('trapani', 'Trapani'),
 ('borussia-dortmund', 'B. Dortmund'),
 ('sporting-braga', 'Sporting Braga'),
 ('fermana', 'Fermana'),
 ('bayer-leverkusen', 'B. Leverkusen'),
 ('sabadell', 'Sabadell'),
 ('xerez', 'Xerez'),
 ('cardiff-city-fc', 'Cardiff City'),
 ('liverpool', 'Liverpool'),
 ('fc-gueugnon', 'FC Gueugnon'),
 ('ac-mantova', 'Mantova'),
 ('stormvogels-telstar', 'SC Telstar'),
 ('castel-sangro', 'Castel Sangro'),
 ('reus', 'Reus Deportiu'),
 ('acireale', 'Acireale'),
 

In [29]:
def team_name_for_link_team_data(t):
    matches = [x[0] for x in matched_teams_team_data if x[1] == t]
    return (matches if len(matches)>0 else [''])[0]
team_df['matched_team'] = team_df.apply(lambda row: team_name_for_link_team_data(row['team']), axis=1)

In [30]:
# Brighton is particularly problematic! Let's see if it gets picked up
team_df[team_df.city=='Brighton']

,team,city,country,stadium,capacity,pitch,matched_team
99,Brighton Hove Alb.,Brighton,England,The American Express Community Stadium,30750,Natural,brighton-amp-hov


In [31]:
team_df[team_df['matched_team']=='crystal-palace-fc']

,team,city,country,stadium,capacity,pitch,matched_team
126,Crystal Palace,Londres,England,Selhurst Park,26309,Natural,crystal-palace-fc


In [32]:
df3 = pd.merge(df2, team_df, how="inner", left_on="home_team_x", right_on="matched_team")
df3

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,link_parts,home_team_x,away_team_x,season_x,home_team_y,away_team_y,result,season_y,round,league,home,away,total_goals,outcome,team,city,country,stadium,capacity,pitch,matched_team
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,"[, match, saarbrucken, stuttgarter-kickers, 1990]",saarbrucken,stuttgarter-kickers,1990,Saarbrücken,Stuttgarter Kickers,1-0,1990,1,2_liga,1,0,1,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,"[, match, saarbrucken, hannover-96, 1990]",saarbrucken,hannover-96,1990,Saarbrücken,Hannover 96,2-0,1990,3,2_liga,2,0,2,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,"[, match, saarbrucken, meppen, 1990]",saarbrucken,meppen,1990,Saarbrücken,Meppen,4-0,1990,5,2_liga,4,0,4,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,"[, match, saarbrucken, blau-weiss-1890-berlin,...",saarbrucken,blau-weiss-1890-berlin,1990,Saarbrücken,Blau-Weiß 1890 Berlin,1-1,1990,7,2_liga,1,1,2,2,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,"[, match, saarbrucken, vfl-osnabruck, 1990]",saarbrucken,vfl-osnabruck,1990,Saarbrücken,VfL Osnabrück,7-1,1990,9,2_liga,7,1,8,3,Saarbrücken,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,saarbrucken
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109238,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,"[, match, bourges, nancy, 1994]",bourges,nancy,1994,Bourges 18,Nancy,2-0,1994,9,ligue_2,2,0,2,3,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109239,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,"[, match, bourges, red-star, 1994]",bourges,red-star,1994,Bourges 18,Red Star,0-1,1994,31,ligue_2,0,1,1,4,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109240,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,"[, match, bourges, sedan, 1994]",bourges,sedan,1994,Bourges 18,Sedan,1-2,1994,10,ligue_2,1,2,3,4,Bourges 18,Bourges,France,NaN,13000,NaN,bourges
109241,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,"[, match, bourges, chamois-niortais, 1994]",bourges,chamois-niortais,1994,Bourges 18,Niort,1-1,1994,18,ligue_2,1,1,2,2,Bourges 18,Bourges,France,NaN,13000,NaN,bourges


# ELO

In [33]:
import pickle
elo_dict = pickle.load(open('additional-data/elo_dict.pkl', 'rb'))
elo = pd.DataFrame.from_dict(elo_dict).transpose()
elo.columns = elo.columns.str.lower()

In [34]:
elo['link_parts'] = elo.index.str.split('/')
elo['home_team'] = elo.apply(lambda row: row['link_parts'][-3], axis=1)
elo['away_team'] = elo.apply(lambda row: row['link_parts'][-2], axis=1)
elo['season'] = elo.apply(lambda row: int(str(row['link_parts'][-1])[:4]), axis=1)
elo['link_index'] = elo.apply(lambda row: str(row['link_parts'][-1])[4:], axis=1)
elo['link'] = elo.apply(lambda row: '/'.join(['','match',row['home_team'],row['away_team'],str(row['season'])]), axis=1)
elo = elo.drop(['link_parts'], axis=1)

In [35]:
elo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132111 entries, https://www.besoccer.com/match/saarbrucken/stuttgarter-kickers/19903487 to https://www.besoccer.com/match/chievo/pisa-calcio/2021103065
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   elo_home    122314 non-null  float64
 1   elo_away    122314 non-null  float64
 2   home_team   132111 non-null  object 
 3   away_team   132111 non-null  object 
 4   season      132111 non-null  int64  
 5   link_index  132111 non-null  object 
 6   link        132111 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 8.1+ MB


In [36]:
df3_teams = set(df3.home_team_x)
# lower case fixes sd-Compostela which was the only non-match
elo_teams = set(elo.home_team.str.lower())
# empty set difference shows that all teams in our data so far (df3) are included in elo
df3_teams.difference(elo_teams)

set()

In [37]:
elo.home_team = elo.home_team.str.lower()

In [38]:
df4 = pd.merge(df3, elo, how="left", on="link")
# clean up - drop some of the columns created by the joins
df4.drop(['link_parts','home_team_y','away_team_y','season_y','team','matched_team','home_team','away_team','season'], axis=1, inplace=True)
df4.rename(columns={'home_team_x':'home_team','away_team_x':'away_team','season_x':'season','home':'home_goals','away':'away_goals'}, inplace=True)
df4

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,season,result,round,league,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,saarbrucken,stuttgarter-kickers,1990,1-0,1,2_liga,1,0,1,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,59.0,3487
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,saarbrucken,hannover-96,1990,2-0,3,2_liga,2,0,2,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,60.0,3507
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,saarbrucken,meppen,1990,4-0,5,2_liga,4,0,4,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,53.0,3533
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,saarbrucken,blau-weiss-1890-berlin,1990,1-1,7,2_liga,1,1,2,2,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,56.0,3553
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,saarbrucken,vfl-osnabruck,1990,7-1,9,2_liga,7,1,8,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,52.0,3575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109242,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,bourges,nancy,1994,2-0,9,ligue_2,2,0,2,3,Bourges,France,NaN,13000,NaN,NaN,NaN,35314
109243,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,bourges,red-star,1994,0-1,31,ligue_2,0,1,1,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35553
109244,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,bourges,sedan,1994,1-2,10,ligue_2,1,2,3,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35322
109245,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,bourges,chamois-niortais,1994,1-1,18,ligue_2,1,1,2,2,Bourges,France,NaN,13000,NaN,NaN,NaN,35411


In [39]:
fulldf = df4.copy()

# Feature engineering

## Newly-promoted teams (and newly-relegated)

Highlight which teams are new in the league this season

In [40]:
# try this for premier_league 2018 as an example
def teams(league,season):
    return set(fulldf[(fulldf['league']==league) & (fulldf['season']==season)].home_team)
pl18_teams = teams('premier_league','2018')
pl17_teams = teams('premier_league','2017')
pl18_teams.difference(pl17_teams)

{'brighton-amp-hov', 'huddersfield-town-fc', 'newcastle-united-fc'}

In [41]:
# we have to connect pairs of leagues so we can look at relegation and promotion between them

league_pairings = [
    ('ligue_1','ligue_2'),
    ('eredivisie','eerste_divisie'),
    ('premier_league','championship'),
    ('primera_division','segunda_division'),
    ('primeira_liga','segunda_liga'),
    ('serie_a','serie_b'),
    ('bundesliga','2_liga')
]

# returns the league that's connected (same country) to the one supplied, and t/f as to whether the league supplied as l is the top league
def pairing(l):
    pair = [p for p in league_pairings if l in p][0]
    return ([t for t in pair if t != l][0], l == pair[0]) 

# eg

In [42]:
def teams_new_this_season(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    return list(this_season.difference(previous_season))

def teams_gone_from_last_season(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    return list(previous_season.difference(this_season))


In [43]:
# returns a 2-tuple of (promoted teams, relegated teams) - i.e. lists of teams in this league this season who arrived by either promotion or relegation
def promoted_teams(league, season):
    previous_season = teams(league,str(int(season)-1))
    this_season = teams(league, season)
    if len(previous_season) == 0:
        return ([],[])
    new_this_season = teams_new_this_season(league, season)
    connected = pairing(league)
    if not connected[1]: # league is NOT the top league so we need to look at relegated teams too
        relegated_teams = teams_gone_from_last_season(connected[0], season)
        promoted_teams = list(set(new_this_season).difference(set(relegated_teams)))
        return (promoted_teams, relegated_teams)
    else:
        return (new_this_season, [])
    

In [44]:
promoted_teams('championship','2000')

(['fulham', 'walsall'],
 ['nottingham-forest-fc', 'blackburn-rovers-fc', 'charlton-athletic-fc'])

In [45]:
leagues = list(set(fulldf.league))
seasons = list(set(fulldf.season))
df_moved_teams = pd.DataFrame(columns=leagues)
df_moved_teams['season'] = seasons
df_moved_teams.set_index('season', inplace=True)
for league in leagues:
    for season in seasons:
        df_moved_teams.at[season,league] = promoted_teams(league, season)
df_moved_teams

,eerste_divisie,ligue_1,segunda_division,premier_league,serie_a,bundesliga,2_liga,ligue_2,championship,primeira_liga,serie_b,segunda_liga,eredivisie,primera_division
season,,,,,,,,,,,,,,
1991,"([], [])","([stade-rennes, monaco, bordeaux, as-cannes, a...","([ue-lleida, albacete, orihuela], [celta, cd-m...","([leeds-united-afc, sheffield-united, sunderla...","([parma-fc, torino-fc, cagliari], [])","([hertha-bsc, wattenscheid-09], [])","([vfb-oldenburg, havelse, schweinfurt], [waldh...","([], [])","([notts-county, bristol-city-fc, bristol-rover...","([salgueiros, gil-vicente, farense, famalicao]...","([], [])","([], [])","([svv, heerenveen], [])","([betis, espanyol, real-burgos-cf], [])"
1996,"([], [])","([guingamp, fc-gueugnon], [])","([sestao-sport-club, ecija, almeria, alaves], ...","([middlesbrough-fc, bolton-wanderers-fc], [])","([piacenza, atalanta, vicenza-calcio, udinese]...","([hansa-rostock, fortuna-dusseldorf], [])","([arminia-bielefeld, lubeck, fc-carl-zeiss-jen...","([], [])","([huddersfield-town-fc, birmingham-city-fc], [...","([fc-felgueiras, campomaiorense], [])","([bologna, reggina, us-avellino, pistoiese], [...","([], [])","([fortuna-sittard, graafschap], [])","([salamanca, merida-cp, rayo-vallecano], [])"
1990,"([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])","([], [])"
2005,"([heracles, fc-oss, go-ahead-eagles, excelsior...","([caen, saint-etienne, istres], [])","([pontevedra-cf, ue-lleida, gimnastic-tarragon...","([crystal-palace-fc, west-bromwich, norwich-ci...","([], [])","([arminia-bielefeld], [])","([sg-dynamo-dresden, rot-weib-erfurt, saarbruc...","([dijon-fco, stade-brestois-29, stade-reims], ...","([brighton-amp-hov, plymouth-argyle, queens-pa...","([vitoria-setubal, penafiel, estoril], [])","([fc-catanzaro, fc-crotone, ac-arezzo, ac-cese...","([sc-olhanense, espinho, varzim, gondomar], [p...","([fc-den-bosch, graafschap], [])","([numancia, getafe, levante], [])"
2006,"([sc-cambuur-leeuwarden, vvv, mvv, fc-zwolle],...","([troyes, nancy], [])","([cd-castellon, rm-castilla, hercules], [numan...","([wigan-athletic, sunderland-afc, west-ham-uni...","([], [])","([msv-duisburg, eintracht-frankfurt], [])","([kickers-offenbach, paderborn, sportfreunde-s...","([valenciennes-afc], [caen, istres, bastia])","([], [crystal-palace-fc, southampton-fc, norwi...","([pacos-ferreira, estrela-amadora, naval], [])","([bologna, atalanta, rimini-calcio, ac-mantova...","([vizela, sporting-covilha, feirense, fc-maia,...","([heracles, sparta-rotterdam], [])","([celta, cadiz, alaves], [])"
2021,"([sc-cambuur-leeuwarden, stormvogels-telstar, ...","([lens, lorient], [])","([sabadell, cartagena, cd-castellon, ud-logron...","([leeds-united-afc, fulham, west-bromwich], [])","([fc-crotone, benevento-calcio, asd-spezia], [])","([arminia-bielefeld], [])","([wurzburger-kickers], [fortuna-dusseldorf, pa...","([dunkerque], [toulouse-fc, amiens-sc])","([], [])","([nacional, farense], [])","([ac-monza-brianza-1912, reggina, vicenza-calc...","([cova-piedade, vizela, feirense, academico-vi...","([], [])","([cadiz, huesca, elche], [])"
1994,"([], [])","([as-cannes, angers-sco, fc-martigues], [])","([real-murcia, leganes, cd-toledo, hercules], ...","([swindon-town, newcastle-united-fc, west-ham-...","([us-cremonese, piacenza, lecce], [])","([msv-duisburg, lokomotive-leipzig, sc-freibur...","([tennis-borussia, rot-weiss-essen, 1860-munch...","([], [])","([stoke-city, bolton-wanderers-fc, west-bromwi...","([vitoria-setubal, uniao-madeira, estrela-amad...","([], [])","([], [])","([vvv, heerenveen, nac-breda], [])","([ue-lleida, valladolid, racing], [])"
2007,"([graafschap, fc-den-bosch, agovv-apeldoorn, v...","([lorient, sedan, valenciennes-afc], [])","([ud-palmas, salamanca, ud-vecindario, ponferr...","([watford-fc, sheffield-united, reading-fc], [])","([catania, torino-fc, atalanta], [])","([bochum, energie-cottbus, alemannia-aachen], [])","([tus-koblenz,

In [46]:
fulldf['home_newly_promoted'] = fulldf.apply(lambda row: row.home_team in df_moved_teams.at[row.season,row.league][0], axis=1)
fulldf['home_newly_relegated'] = fulldf.apply(lambda row: row.home_team in df_moved_teams.at[row.season,row.league][1], axis=1)
fulldf['away_newly_promoted'] = fulldf.apply(lambda row: row.away_team in df_moved_teams.at[row.season,row.league][0], axis=1)
fulldf['away_newly_relegated'] = fulldf.apply(lambda row: row.away_team in df_moved_teams.at[row.season,row.league][1], axis=1)
fulldf

,link,referee,home_yellow,home_red,away_yellow,away_red,date_dt,home_team,away_team,season,result,round,league,home_goals,away_goals,total_goals,outcome,city,country,stadium,capacity,pitch,elo_home,elo_away,link_index,home_newly_promoted,home_newly_relegated,away_newly_promoted,away_newly_relegated
0,/match/saarbrucken/stuttgarter-kickers/1990,Hans-Jürgen Weber,0.0,0.0,3.0,0.0,1989-07-29 15:00:00,saarbrucken,stuttgarter-kickers,1990,1-0,1,2_liga,1,0,1,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,59.0,3487,False,False,False,False
1,/match/saarbrucken/hannover-96/1990,Eugen Strigel,0.0,0.0,3.0,0.0,1989-08-12 15:00:00,saarbrucken,hannover-96,1990,2-0,3,2_liga,2,0,2,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,56.0,60.0,3507,False,False,False,False
2,/match/saarbrucken/meppen/1990,Siegbert Rubel,0.0,0.0,3.0,0.0,1989-08-23 19:30:00,saarbrucken,meppen,1990,4-0,5,2_liga,4,0,4,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,53.0,3533,False,False,False,False
3,/match/saarbrucken/blau-weiss-1890-berlin/1990,Wilfried Heitmann,0.0,0.0,0.0,0.0,1989-09-02 15:00:00,saarbrucken,blau-weiss-1890-berlin,1990,1-1,7,2_liga,1,1,2,2,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,56.0,3553,False,False,False,False
4,/match/saarbrucken/vfl-osnabruck/1990,Karl Gangkofer,1.0,0.0,2.0,0.0,1989-09-17 15:00:00,saarbrucken,vfl-osnabruck,1990,7-1,9,2_liga,7,1,8,3,Saarbrücken,Germany,Stadion Ludwigspark,35286,Natural,57.0,52.0,3575,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109242,/match/bourges/nancy/1994,Marc Flosi,0.0,0.0,0.0,1.0,1993-09-01 00:00:00,bourges,nancy,1994,2-0,9,ligue_2,2,0,2,3,Bourges,France,NaN,13000,NaN,NaN,NaN,35314,False,False,False,False
109243,/match/bourges/red-star/1994,,0.0,0.0,0.0,1.0,1994-03-05 00:00:00,bourges,red-star,1994,0-1,31,ligue_2,0,1,1,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35553,False,False,False,False
109244,/match/bourges/sedan/1994,Stéphane Moulin,0.0,0.0,0.0,1.0,1993-09-11 00:00:00,bourges,sedan,1994,1-2,10,ligue_2,1,2,3,4,Bourges,France,NaN,13000,NaN,NaN,NaN,35322,False,False,False,False
109245,/match/bourges/chamois-niortais/1994,Hervé Bayet,0.0,0.0,0.0,0.0,1993-10-27 00:00:00,bourges,chamois-niortais,1994,1-1,18,ligue_2,1,1,2,2,Bourges,France,NaN,13000,NaN,NaN,NaN,35411,False,False,False,False


In [47]:
# drop the items missing elo
print(fulldf.size)
fulldf.dropna(subset=['elo_home','elo_away'],inplace=True)
print(fulldf.size)

3168163
2959334


# Winning / losing streaks